In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import nltk
import re, string, unicodedata
from nltk.corpus import stopwords
#from textblob import TextBlob
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

## Dropping unnecessary columns

In [2]:
#df = pd.read_csv('Datasets\BL-Flickr-Images-Book.csv')
data=pd.read_csv('D:\\R-Projects\\Amazon_Unlocked_Mobile.csv')
data.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [3]:
data.columns

Index(['Product Name', 'Brand Name', 'Price', 'Rating', 'Reviews',
       'Review Votes'],
      dtype='object')

In [4]:
data.shape

(413840, 6)

In [5]:
#let's drop the neutral review which is always 3 so that we can remain with either negative or positive review
data.dropna(inplace=True)
data[data['Rating'] != 3]
#Now we are defining the positive rating as those with greater than 3
data['Positively Rated'] = np.where(data['Rating'] > 3, 1, 0)
data['Negatively Rated'] = np.where(data['Rating'] < 3, 4, 5)
data.head(7)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated,Negatively Rated
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,1,5
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,1,5
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,1,5
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,1,5
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,1,5
5,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,1,I already had a phone with problems... I know ...,1.0,0,4
6,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,The charging port was loose. I got that solder...,0.0,0,4


In [6]:
data['Positively Rated'].mean()

0.6899487041440472

In [7]:
data['Negatively Rated'].mean()

4.767888495072308

The above mean is higher and it shows that reviews with higher mean are said to be more biased.

In [10]:
#Now we want to do serious pre-processing

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    #words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

#Steemming and Lemmatization
def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

#stems, lemmas = stem_and_lemmatize(words)
#print('Stemmed:\n', stems)
#print('\nLemmatized:\n', lemmas)


# ---------------   Cleaning   ------------------
def clean_text(text):
    wording = nltk.word_tokenize(text)
    words = normalize(wording)
    string_text = ' '.join(words)
    return string_text

# ---------------   Sentiment   ------------------
def get_text_sentiment(text):
    # create TextBlob object of passed text 
    analysis = TextBlob(clean_text(text)) 
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'    
    
#---------------------- TextBlob Feature Extractions -----------------
#Function to extract features from text
def textBlob_feature_extraction(text): 
        blob = TextBlob(text)
        return blob.noun_phrases
    
    
#----------------------  extract Sentences  -------------------------    
def sentances(text):
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    return sent_detector.tokenize(text.strip())

In [9]:
#-----------------  Text Cleaning & Sentiment Extraction --------------------
cleaned_reviews =[]
sentiment= []
for reviews in data['Reviews']:
    cleaned_reviews.append(clean_text(reviews))
    sentiment.append(get_text_sentiment(reviews))
    
features_Dataset = pd.DataFrame()
features_Dataset['Product Name'] = data['Product Name']
features_Dataset['Reviews'] = data['Reviews']
features_Dataset['Cleaned_Reviews'] = cleaned_reviews
features_Dataset['Sentiment'] = sentiment


# Extracting Features from each review using TextBlob & Spacy *
    
#--------------------- Text Blob -------------------------
features = []
for reviews in features_Dataset['Cleaned_Reviews']:
    features.append(textBlob_feature_extraction(reviews))
#adding Extracted features to dataset
features_Dataset["TextBlob_Features"] = features

#--------------------- Spacy -----------------------------
nlp = spacy.load('en')

feature_spacy = []
for review in nlp.pipe(features_Dataset['Cleaned_Reviews']):
    chunks = [(chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == 'NOUN']
    feature_spacy.append(','.join(chunks))

features_Dataset['Spacy_features']= feature_spacy

#---------------------------------------------------------
features_Dataset.head()
#https://www.kaggle.com/ahtxham/amazon-dataset-preprocessing

NameError: name 'TextBlob' is not defined

How many brands do we have in our dataset?

In [11]:
len(list(set(data['Brand Name'])))

378

In [12]:
len(list(set(data['Product Name'])))

3675

In [13]:
#Let's get the summary of products
pivot = pd.pivot_table(data,
            values = ['Rating', 'Review Votes'],
            index = ['Brand Name'], 
                       columns= [],
                       aggfunc=[np.sum, np.mean, np.count_nonzero, np.std], 
                       margins=True).fillna('')
pivot.head(10)

sum                   mean               \
                       Rating Review Votes    Rating Review Votes   
AMM Global Enterprises     44          1.0  4.888889     0.111111   
ARGOM TECH                  6          0.0  3.000000     0.000000   
ASUS                     2092        838.0  4.470085     1.790598   
ASUS Computers           2852        831.0  4.028249     1.173729   
AT&T                        5          0.0  5.000000     0.000000   
ATT                       160         25.0  3.636364     0.568182   
Acer                       68         27.0  3.090909     1.227273   
Aeku                       10          8.0  5.000000     4.000000   
AeroAntenna                 5          0.0  5.000000     0.000000   
Alcatel                  5574       3174.0  4.039130     2.300000   

                       count_nonzero                    std               
                              Rating Review Votes    Rating Review Votes  
AMM Global Enterprises             9          1.0  0.333333     0.333333  
ARGOM TECH                         2          0.0   2.82843            0  
ASUS                             468        185.0   1.08383      4.81999  
ASUS Computers                   708        228.0    1.3836      6.68461  
AT&T                               1          0.0                         
ATT                               44         17.0   1.79263     0.974028  
Acer                              22         14.0   1.71573      1.37778  
Aeku                               2          1.0         0      5.65685  
AeroAntenna                        1          0.0                         
Alcatel                         1380        777.0   1.36885      8.09721

In [14]:
#Can we get to know which brands are top performing?
pivot = pd.pivot_table(data,
            values = ['Rating', 'Review Votes'],
            index =  ['Brand Name'],
                       columns= [],
                       aggfunc=[np.sum, np.mean, np.count_nonzero, np.std], 
                       margins=True, fill_value=0).sort_values(by=('count_nonzero', 'Rating'), ascending=False).fillna('')
top_10_brands = pivot.reindex().head(n=11)
top_10_brands

sum                   mean              count_nonzero  \
             Rating Review Votes    Rating Review Votes        Rating   
All         1278794       492982  3.824888     1.474515        334335   
Samsung      250452        96057  3.973032     1.523795         63038   
BLU          226085        54798  3.821069     0.926143         59168   
Apple        220286       112211  3.926597     2.000160         56101   
LG            83266        22929  3.848493     1.059762         21636   
BlackBerry    61892        21114  3.750121     1.279326         16504   
Nokia         61833        25684  3.824879     1.588767         16166   
Motorola      49564        23107  3.811736     1.777051         13003   
HTC           42873        12777  3.474030     1.035329         12341   
CNPGD         38233        20151  3.107869     1.638026         12302   
OtterBox      34556         2268  4.385279     0.287817          7880   

                             std               
           Review Votes   Rating Review Votes  
All              101271   1.5412      9.21733  
Samsung           18221  1.47913      9.95184  
BLU               15182  1.48947      5.45117  
Apple             18355  1.57473      13.2452  
LG                 5879  1.53039      5.09583  
BlackBerry         4058  1.59666      7.72454  
Nokia              5491  1.48911        7.865  
Motorola           4392  1.52564      15.2721  
HTC                3170  1.65743      5.51806  
CNPGD              3502  1.61922      8.96474  
OtterBox            741   1.1622      2.62002

In [15]:
#dropped rows having NaN values
data_df = data.dropna()

In [16]:
#What's the correlation between 
corr_matrix = data_df.corr()
corr_matrix["Rating"].sort_values(ascending = False)

Rating              1.000000
Positively Rated    0.925830
Negatively Rated    0.915473
Price               0.073948
Review Votes       -0.046526
Name: Rating, dtype: float64

From the above output, we can discover that Positively Rated brands have high corelation with Rating as compared to Negatively Rated.
Price has minimal correlation while Review Votes lack any relationship with Rating.

In [17]:
corr_matrix = data_df.corr()
corr_matrix["Price"].sort_values(ascending = False)

Price               1.000000
Rating              0.073948
Positively Rated    0.073898
Negatively Rated    0.054158
Review Votes        0.022164
Name: Price, dtype: float64

In [18]:
#The shuffle library works like indexing
from sklearn.utils import shuffle
data_df = shuffle(data) #Shuffle Data
data_df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated,Negatively Rated
380280,Samsung Note 2 I317 16GB Unlocked GSM 4G LTE Q...,Samsung,159.95,5,Great phone. It worked as stated. The scratche...,0.0,1,5
290385,Pantech Breeze C520 Unlocked GSM Flip Phone,Pantech,13.95,4,thank you..i found a new phone just like my ol...,1.0,1,5
262150,Nokia 8600 Unlocked Phone (Black),Nokia,1169.10,1,Fake,1.0,0,4
100332,Blackberry Torch 2 9810 Unlocked Phone with 1....,BlackBerry,77.49,5,excelete rpoducto,0.0,1,5
233627,LG Optimus G E970 16GB Unlocked GSM 4G LTE Qua...,LG,449.00,5,Excellent,0.0,1,5


In [19]:
#We can then decide to reset our index by writing the function below
#reset_index
data = data_df.reset_index(drop=True)
data.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated,Negatively Rated
0,Samsung Note 2 I317 16GB Unlocked GSM 4G LTE Q...,Samsung,159.95,5,Great phone. It worked as stated. The scratche...,0.0,1,5
1,Pantech Breeze C520 Unlocked GSM Flip Phone,Pantech,13.95,4,thank you..i found a new phone just like my ol...,1.0,1,5
2,Nokia 8600 Unlocked Phone (Black),Nokia,1169.10,1,Fake,1.0,0,4
3,Blackberry Torch 2 9810 Unlocked Phone with 1....,BlackBerry,77.49,5,excelete rpoducto,0.0,1,5
4,LG Optimus G E970 16GB Unlocked GSM 4G LTE Qua...,LG,449.00,5,Excellent,0.0,1,5


Let's advice the Apple Company on their phones

In [20]:
#The data of apple phones in the dataset
data_apple = data.loc[data['Brand Name'].isin(['Apple'])]
pivot = pd.pivot_table(data_apple,
        values = ['Rating', 'Review Votes'],
        index =  ['Product Name'],
                   columns= [],
                   aggfunc=[np.sum, np.mean, np.count_nonzero], 
                   margins=True, fill_value=0).sort_values(by=('count_nonzero', 'Rating'), ascending=False).fillna('')
topmost_prods = pivot.reindex().head(n=30)
topmost_prods = topmost_prods.reset_index()
topmost_prods
data_apple.head(8)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated,Negatively Rated
13,Apple iPhone 3GS A1303 16GB GSM Unlocked Smart...,Apple,39.00,5,Excelente,0.0,1,5
14,"Apple iPhone 5s T-Mobile Cellphone, 16GB, Spac...",Apple,168.77,1,I unlock this phone on the site [...] for use ...,1.0,0,4
21,"Apple iPhone 6S Plus Unlocked Smartphone, 32 G...",Apple,749.99,5,AWESOME!!!! I haven't figured out the technolo...,3.0,1,5
31,Apple iPhone 5c 16GB (Yellow) - AT&T,Apple,248.00,4,"it has a glitch that drops wifi, and you can't...",0.0,1,5
33,Apple iPhone 5s 32GB (Gold) - AT&T,Apple,54.34,5,grate phone fore the price. got it for gift fo...,0.0,1,5
37,Apple iPhone 7 Unlocked Phone 128 GB - US Vers...,Apple,829.95,1,This is fake phone not the original iPhone 7 I...,2.0,0,4
38,Apple iPhone 6s 64 GB International Warranty U...,Apple,689.95,5,Good condition as posted. Very pleased!,0.0,1,5
41,Apple iPhone SE Unlocked Phone - 64 GB Retail ...,Apple,500.00,5,I got this phone for my wife. It is a great up...,0.0,1,5


# Modeling starts here with Bag-of-Words Model

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['Reviews'], data['Positively Rated'], random_state = 0)

In [22]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer().fit(X_train)

In [23]:
vect.get_feature_names()[::3000]

['00',
 '85f',
 'appworld',
 'bothare',
 'coke',
 'decidir',
 'eggs',
 'fingerprint3',
 'h860',
 'intelligible',
 'locals',
 'murtadha',
 'owner',
 'proble3m',
 'reparaciones',
 'sharp',
 'stunt',
 'tracy',
 'vqlhccjh434']

In [24]:
len(vect.get_feature_names())

56541

In [25]:
# transform the documents in the training data to a document-term matrix
X_train_vectorized = vect.transform(X_train)
X_train_vectorized

<250751x56541 sparse matrix of type '<class 'numpy.int64'>'
	with 6831683 stored elements in Compressed Sparse Row format>

In [26]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
from sklearn.metrics import roc_auc_score

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

In [27]:
# get the feature names as numpy array
feature_names = np.array(vect.get_feature_names())

# Sort the coefficients from the model
sorted_coef_index = model.coef_[0].argsort()

# Find the 10 smallest and 10 largest coefficients
# The 10 largest coefficients are being indexed using [:-11:-1] 
# so the list returned is in order of largest to smallest
print('Smallest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs: 
['stylist' 'false' 'worst' 'unsatisfied' 'horribly' 'raymond' 'mony'
 'worthless' 'unusable' 'saler']

Largest Coefs: 
['excelent' 'excelente' 'perfecto' 'exelente' 'excellent' 'awsome'
 'loving' 'superb' 'eight' 'exellent']



In [ ]:
#Tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
vect = TfidfVectorizer(min_df = 5).fit(X_train)
len(vect.get_feature_names())

In [ ]:
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

In [28]:
feature_names = np.array(vect.get_feature_names())

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

print('Smallest Tfidf: \n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest Tfidf: \n{}\n'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Smallest Tfidf: 
['junction' 'overreacted' 'override' 'overriding' 'overrode' 'overruled'
 'overs' 'oversaes' 'oversampling' 'oversaturated']

Largest Tfidf: 
['the' 'to' 'and' 'it' 'you' 'phone' 'is' 'of' 'on' 'this']



In [29]:
sorted_coef_index = model.coef_[0].argsort()

print('Smallest coef: \n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest coef: \n{}\n'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest coef: 
['stylist' 'false' 'worst' 'unsatisfied' 'horribly' 'raymond' 'mony'
 'worthless' 'unusable' 'saler']

Largest coef: 
['excelent' 'excelente' 'perfecto' 'exelente' 'excellent' 'awsome'
 'loving' 'superb' 'eight' 'exellent']



In [30]:
# These reviews are treated the same by our current model

print(model.predict(vect.transform(['Not an issue, phone is working', 
                                   'an issue, phone is not working'])))

[0 0]


In [31]:
# n-grams
# Fit the CountVectorizer to the training data specifiying a minimum 
# document frequency of 5 and extracting 1-grams and 2-grams
vect = CountVectorizer(min_df = 5, ngram_range = (1,2)).fit(X_train)
X_train_vectorized = vect.transform(X_train)
len(vect.get_feature_names())

217247

In [32]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))
print('AUC: ', roc_auc_score(y_test, predictions))

NameError: name 'roc_auc_score' is not defined

In [ ]:
feature_names = np.array(vect.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coef: \n{}\n'.format(feature_names[sorted_coef_index][:10]))
print('Largest Coef: \n{}\n'.format(feature_names[sorted_coef_index][:-11:-1]))

In [ ]:
print(model.predict(vect.transform(['not an issue, phone is working',
                                   'an issue, phone is not working'])))
#https://github.com/susanli2016/NLP-with-Python/blob/master/Amazon%20Reviews.ipynb

## Cleaning columns using the `.apply` function

In [ ]:
unwanted_characters = ['[', ',', '-']

def clean_dates(item):
    dop= str(item.loc['Date of Publication'])
    
    if dop == 'nan' or dop[0] == '[':
        return np.NaN
    
    for character in unwanted_characters:
        if character in dop:
            character_index = dop.find(character)
            dop = dop[:character_index]
    
    return dop

df['Date of Publication'] = df.apply(clean_dates, axis = 1)

In [ ]:
df.head()

In [ ]:
#alternate way of cleaning Date of Publication
#run cell to see output
unwanted_characters = ['[', ',', '-']

def clean_dates(dop):
    dop = str(dop)
    if dop.startswith('[') or dop == 'nan':
        return 'NaN'
    for character in unwanted_characters:
        if character in dop:
            character_index = dop.find(character)
            dop = dop[:character_index]
    return dop

df['Date of Publication'] = df['Date of Publication'].apply(clean_dates)
df.head()

In [ ]:
def clean_author_names(author):
    
    author = str(author)
    
    if author == 'nan':
        return 'NaN'
    
    author = author.split(',')

    if len(author) == 1:
        name = filter(lambda x: x.isalpha(), author[0])
        return reduce(lambda x, y: x + y, name)
    
    last_name, first_name = author[0], author[1]

    first_name = first_name[:first_name.find('-')] if '-' in first_name else first_name
    
    if first_name.endswith(('.', '.|')):
        parts = first_name.split('.')
        
        if len(parts) > 1:
            first_occurence = first_name.find('.')
            final_occurence = first_name.find('.', first_occurence + 1)
            first_name = first_name[:final_occurence]
        else:
            first_name = first_name[:first_name.find('.')]
    
    last_name = last_name.capitalize()
    
    return f'{first_name} {last_name}'


df['Author'] = df['Author'].apply(clean_author_names)

In [ ]:
df.head()

In [ ]:
def clean_title(title):
    
    if title == 'nan':
        return 'NaN'
    
    if title[0] == '[':
        title = title[1: title.find(']')]
        
    if 'by' in title:
        title = title[:title.find('by')]
    elif 'By' in title:
        title = title[:title.find('By')]
        
    if '[' in title:
        title = title[:title.find('[')]

    title = title[:-2]
        
    title = list(map(str.capitalize, title.split()))
    return ' '.join(title)
    
df['Title'] = df['Title'].apply(clean_title)
df.head()

## Using `.str` methods to clean columns

In [ ]:
#4157862 and 4159587
df.loc[4159587]

In [ ]:
pub = df['Place of Publication']
df['Place of Publication'] = np.where(pub.str.contains('London'), 'London',
    np.where(pub.str.contains('Oxford'), 'Oxford',
        np.where(pub.eq('Newcastle upon Tyne'),
            'Newcastle-upon-Tyne', df['Place of Publication'])))

In [ ]:
df.head()

## Cleaning entire dataset

In [ ]:
!more Datasets\\university_towns.txt 

In [ ]:
university_towns = []

with open('Datasets\\university_towns.txt', 'r') as file:
    items = file.readlines()
    states = list(filter(lambda x: '[edit]' in x, items))
    
    for index, state in enumerate(states):
        start = items.index(state) + 1
        if index == 49: #since 50 states
            end = len(items)
        else:
            end = items.index(states[index + 1])
            
        pairs = map(lambda x: [state, x], items[start:end])
        university_towns.extend(pairs)
        
towns_df = pd.DataFrame(university_towns, columns = ['State', 'RegionName'])
towns_df.head()

In [ ]:
def clean_up(item):
    if '(' in item:
        return item[:item.find('(') - 1]
    
    if '[' in item:
        return item[:item.find('[')]
    

towns_df =  towns_df.applymap(clean_up)
towns_df.head()

## Renaming columns and skipping rows

In [ ]:
olympics_df = pd.read_csv('Datasets\olympics.csv')
olympics_df.head()

In [ ]:
olympics_df = pd.read_csv('Datasets\olympics.csv', skiprows = 1, header = 0)
olympics_df.head()

In [ ]:
new_names =  {'Unnamed: 0': 'Country',
              '? Summer': 'Summer Olympics',
              '01 !': 'Gold',
              '02 !': 'Silver',
              '03 !': 'Bronze',
              '? Winter': 'Winter Olympics',
              '01 !.1': 'Gold.1',
              '02 !.1': 'Silver.1',
              '03 !.1': 'Bronze.1',
              '? Games': '# Games', 
              '01 !.2': 'Gold.2',
              '02 !.2': 'Silver.2',
              '03 !.2': 'Bronze.2'}

olympics_df.rename(columns = new_names, inplace = True)

In [ ]:
olympics_df.head()